In [1]:
%pip install torch torchvision

DEPRECATION: amazon-textract-overlayer 0.0.11 has a non-standard dependency specifier Pillow>=9.2.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-overlayer or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: amazon-textract-overlayer 0.0.11 has a non-standard dependency specifier pypdf>=2.5.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of amazon-textract-overlayer or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchvision.models as models

In [3]:
resnet18 = models.resnet18(pretrained=True)

/Users/zaynpatel/opt/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/zaynpatel/opt/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Prepare images and "Train" 

In [13]:
# Write a custom class to add image identifiers to vector representations for better analysis
import os
from PIL import Image
from torch.utils.data import Dataset

class UnderstandImages(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir # this will be the dictionary passed in the custom class UnderstandImages {'className' : 'file/directory/for/class'}
        self.transform = transform # transform is the same as the block below - we want to begin converting it into tensor-like format
        self.img_names = [] # initializing this as an empty list because I want to iterate over each class directory and list their files

        for classname, directoryname in self.img_dir.items():
            image_names = os.listdir(directoryname) # list the items in the directory - does this list the items on every loop?
            for img_name in image_names:
                img_path = os.path.join(directoryname, img_name) # join the directory with the image names
                self.img_names.append((img_path, classname))
    
    def __len__(self):
        return len(self.img_names) 

    # override this class so I can get image and filename
    def __getitem__(self, idx):
        img_path, classname = self.img_names[idx] # we need to tuple unpack because we get two items in self.img_names based on our append
        image = Image.open(img_path) # convert raw images into tensors
        if self.transform: # apply the transform to each image 
            image = self.transform(image)

        filename = os.path.basename(img_path) # want to see the images in the output filenames.txt as well
    
        return image, classname, filename # return the image and classname because we want to use these for the labeling later on

In [14]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), # crop to the center since we assume most of the important features are in the center
    transforms.ToTensor(), # converts a PIL image, like the one defined in the above code block to a PyTorch tensor or np.ndarray
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalize based on the values of the resnet when it was originally trained
])  

In [15]:
import torch
resnet18.fc = torch.nn.Identity() # remove the final classification layer of the NN so I can just get the feature vectors

In [31]:
from torch.utils.data import DataLoader
import numpy as np # need a way to persist data so use np array to save items to .npy files too

img_dirs = {
    'After Room 126' : '/Users/zaynpatel/vision/visionLLM/CompRobo-VisionLLM/featureExtraction/train/AFTER_126',
    'MAC Entrance' : '/Users/zaynpatel/vision/visionLLM/CompRobo-VisionLLM/featureExtraction/train/MAC_Entrance',
    'Near Elevator' : '/Users/zaynpatel/vision/visionLLM/CompRobo-VisionLLM/featureExtraction/train/QEA_TO_ELEVATOR'
}

dataset = UnderstandImages(img_dir = img_dirs, transform=transform) # use the custom defined class to get filename in the dataloader
dataloader = DataLoader(dataset, batch_size=9, shuffle=False, num_workers=0) # pass in dataset of images, batch_size which is the number of images getting passed to the model, num_workers is number working in parallel on machine

resnet18.eval()

featuresList = []
filenamesList = []
classnameList = []

with torch.no_grad():
    for inputs, classnames, filenames in dataloader: # based on the class we defined, we know there are two inputs to dataloader so we unpack those here for use
        features = resnet18(inputs) # apply the resnet on the images
        # always confirm that we're unpacking the same number of values as we are the 
        for feature, filename, classname in zip(features, filenames, classnames): # zip all output feature vectors and filenames together so we can visualize and compare after
            features_np = feature.detach().cpu().numpy().flatten() # error: did features.detch() instead of feature.detach() which meant ...
            featuresList.append(features_np)
            filenamesList.append(filename)
            classnameList.append(classname) 

featuresArray = np.array(featuresList)
np.save('data.npy', featuresArray)

with open('filenames.txt', 'w') as f:
    for filename, classname in zip(filenamesList, classnameList):
        f.write(f"{filename}, {classname}\n")

In [39]:
from sklearn.neighbors import NearestNeighbors

data = np.load('data.npy', allow_pickle=True) # we said pickle is used because ...
with open('filenames.txt', 'r') as f:
    filename = f.readlines()
    classnames_list = []
    img_file_list = []
    for files in filename:
        # definitely could do this in a list comprehension
        split_files = files.split(',')
        img_files = split_files[0]
        classname_split = split_files[1]
        img_file_list.append(img_files)
        classnames_list.append(classname_split)

nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(data) # ball_tree eliminates datapoints that are too far from the query point, making search faster
distances, indices = nbrs.kneighbors(data)

for i, neighbor_indices in enumerate(indices):
    # Skip the first neighbor if it's the feature itself - it is in this case   
    neighbor_indices = neighbor_indices[1:] if len(neighbor_indices) > 1 else neighbor_indices
    # Get the class names of the neighbors
    neighbor_classnames = [classnames_list[idx] for idx in neighbor_indices] # iterate through the indices which are pairs like [28, 3] and retrieve this corresponding index from the lists where these came from.
    neighbor_filenames = [img_file_list[idx] for idx in neighbor_indices]
    # Get the actual class name for the test feature
    actual_classname = classnames_list[i]
    actual_image = img_file_list[i]

    print(f"Actual class: {actual_classname}, Actual image: {actual_image}")
    print(f"Nearest neighbors' classes: {neighbor_classnames}, Images: {neighbor_filenames}")

#print(f'these are distances {distances}, and indices {indices}')

[30  1]
[' Near Elevator\n', ' After Room 126\n']
Actual class:  After Room 126
, Actual image: after126_4.JPG
Nearest neighbors' classes: [' Near Elevator\n', ' After Room 126\n'], Images: ['IMG_3773.JPG', 'after126_5.JPG']
[1 0]
[' After Room 126\n', ' After Room 126\n']
Actual class:  After Room 126
, Actual image: after126_5.JPG
Nearest neighbors' classes: [' After Room 126\n', ' After Room 126\n'], Images: ['after126_5.JPG', 'after126_4.JPG']
[28  3]
[' Near Elevator\n', ' After Room 126\n']
Actual class:  After Room 126
, Actual image: after126_7.JPG
Nearest neighbors' classes: [' Near Elevator\n', ' After Room 126\n'], Images: ['IMG_3774.JPG', 'after126_6.JPG']
[3 2]
[' After Room 126\n', ' After Room 126\n']
Actual class:  After Room 126
, Actual image: after126_6.JPG
Nearest neighbors' classes: [' After Room 126\n', ' After Room 126\n'], Images: ['after126_6.JPG', 'after126_7.JPG']
[31  5]
[' Near Elevator\n', ' After Room 126\n']
Actual class:  After Room 126
, Actual image: 